In [1]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/Smart_PDF_QA_Sample.pdf


In [2]:
!pip install -q langchain langchain-community langchain-google-genai pypdf faiss-cpu google-generativeai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00


In [3]:
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

import os

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter


loader = PyPDFLoader("/kaggle/input/dataset/Smart_PDF_QA_Sample.pdf")  # Replace with your PDF filename
pages = loader.load()


splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.split_documents(pages)


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS


embedding = GoogleGenerativeAIEmbeddings(api_key=GOOGLE_API_KEY, model="models/embedding-001")

db = FAISS.from_documents(docs, embedding)


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=GOOGLE_API_KEY, temperature=0.3)


In [7]:
from langchain.chains.question_answering import load_qa_chain


chain = load_qa_chain(llm, chain_type="stuff")

# Example query you want to ask about the PDF
query = "Summarize the PDF content in simple language"

# Perform a similarity search to get relevant docs
relevant_docs = db.similarity_search(query)

# Get the answer from the chain
response = chain.run(input_documents=relevant_docs, question=query)
print("📄 Answer:", response)


/tmp/ipykernel_13/2665130334.py:4: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")
/tmp/ipykernel_13/2665130334.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(input_documents=relevant_docs, question=query)


📄 Answer: This document is an introduction to Artificial Intelligence (AI). AI is about making machines that can think and learn like humans. These machines can do things like understand speech, make decisions, and translate languages.

There are different types of AI:
*   Narrow AI: Good at one specific task.
*   General AI: AI with human-like intelligence (but doesn't exist yet).
*   Super AI: AI that is much smarter than humans.

AI is used in many areas, like healthcare, finance, transportation, and customer service. AI can make things more efficient, automate tasks, reduce errors, and be available all the time. However, there are also challenges like ethical concerns, job losses, privacy issues, and bias in AI.
